In [1]:
import string
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, RepeatVector
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data_path= '/kaggle/input/fra-eng/fra.txt'
with open(data_path,'r',encoding='utf-8') as f :
	lines=f.read()

In [3]:
# split the text into sentences
def to_lines(text):
	sents=text.strip().split('\n')
	sents=[i.split('\t') for i in sents]
	return sents

In [4]:

fra_eng=to_lines(lines)
fra_eng[:5]


[['Go.', 'Va !'],
 ['Hi.', 'Salut !'],
 ['Run!', 'Cours\u202f!'],
 ['Run!', 'Courez\u202f!'],
 ['Who?', 'Qui ?']]

In [5]:
fra_eng = np.array(fra_eng)
fra_eng[:5]

array([['Go.', 'Va !'],
       ['Hi.', 'Salut !'],
       ['Run!', 'Cours\u202f!'],
       ['Run!', 'Courez\u202f!'],
       ['Who?', 'Qui ?']], dtype='<U349')

In [6]:
fra_eng.shape

(167130, 2)

In [7]:
fra_eng=fra_eng[:100000,:]
fra_eng=fra_eng[:,[0,1]]
fra_eng[:5]

array([['Go.', 'Va !'],
       ['Hi.', 'Salut !'],
       ['Run!', 'Cours\u202f!'],
       ['Run!', 'Courez\u202f!'],
       ['Who?', 'Qui ?']], dtype='<U349')

In [8]:
#data cleaning
fra_eng[:,0]=[s.translate(str.maketrans('','', string.punctuation)) for s in fra_eng[:,0]]
fra_eng[:,1]=[s.translate(str.maketrans('','', string.punctuation)) for s in fra_eng[:,1]]
fra_eng[:5]

array([['Go', 'Va '],
       ['Hi', 'Salut '],
       ['Run', 'Cours\u202f'],
       ['Run', 'Courez\u202f'],
       ['Who', 'Qui ']], dtype='<U349')

In [9]:
# Convert text to lowercase
for i in range(len(fra_eng)):
	fra_eng[i,0]=fra_eng[i,0].lower()
	fra_eng[i,1]=fra_eng[i,1].lower()
fra_eng

array([['go', 'va '],
       ['hi', 'salut '],
       ['run', 'cours\u202f'],
       ...,
       ['stop sucking up to the teacher',
        'arrête de faire de la lèche au prof '],
       ['stop sucking up to the teacher',
        'arrêtez de faire de la lèche au prof '],
       ['stop sucking up to the teacher',
        'arrête de faire de la lèche à la prof ']], dtype='<U349')

In [10]:
# Text to sequence conversion
# function to build a tokenizer
def tokenization(lines):
	tokenizer=Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [11]:
#prepare english tokenizer
eng_tokenizer=tokenization(fra_eng[:,0])
eng_vocab_size=len(eng_tokenizer.word_index)+1
eng_length=20
print('English Vocabulary Size:%d' %eng_vocab_size)

English Vocabulary Size:9061


In [12]:
#prepare french tokenizer
fra_tokenizer=tokenization(fra_eng[:,1])
fra_vocab_size=len(fra_tokenizer.word_index)+1
fra_length=20
print('French Vocabulary Size:%d' %fra_vocab_size)

French Vocabulary Size:21443


In [13]:
def encode_sequences(tokenizer, length, lines):
    # Integer encoded sequences
    seq = tokenizer.texts_to_sequences(lines)
    # Pad sequences with 0 values
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq  # Corrected indentation

In [14]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(fra_eng,test_size=0.2,random_state=12)

In [15]:
#prepare training data
trainX=encode_sequences(fra_tokenizer,fra_length,train[:,1])
trainY=encode_sequences(eng_tokenizer,eng_length,train[:,0])
#prepare validation data
testX=encode_sequences(fra_tokenizer,fra_length,test[:,1])
testY=encode_sequences(eng_tokenizer,eng_length,test[:,0])

In [16]:
# Define model
def define_model(in_vocab, out_vocab, in_timesteps, out_timesteps, units):
    model = Sequential([
        Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True),
        LSTM(units),
        RepeatVector(out_timesteps),
        LSTM(units, return_sequences=True),
        Dense(out_vocab, activation='softmax')
    ])
    return model

In [17]:
model = define_model(fra_vocab_size, eng_vocab_size, fra_length, eng_length, 512)
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss='sparse_categorical_crossentropy')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
# Training the model model
epochs = 50
history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=epochs, batch_size=64, validation_split=0.2)

Epoch 1/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 36s 31ms/step - loss: 1.9112 - val_loss: 1.4493
Epoch 2/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 1.4310 - val_loss: 1.3857
Epoch 3/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 1.3687 - val_loss: 1.3206
Epoch 4/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 1.2909 - val_loss: 1.2409
Epoch 5/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 1.2138 - val_loss: 1.1790
Epoch 6/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 1.1506 - val_loss: 1.1214
Epoch 7/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 1.0940 - val_loss: 1.0799
Epoch 8/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 1.0315 - val_loss: 1.0270
Epoch 9/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 0.9786 - val_loss: 0.9827
Epoch 10/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 0.9278 - val_loss: 0.9477
Epoch 11/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 0.8792 - val_loss: 0.9111
Epoch 12

In [28]:
preds=np.argmax(model.predict(testX), axis=-1)
preds

InternalError: Graph execution error:

Detected at node sequential_1/lstm_1_2/CudnnRNNV3 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/pyt

In [24]:
# Getting the words as the model predicts the results in a numerical format
def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == n:
            return word
    return None

In [25]:
preds_text = []
for i in preds:
    temp = []
    for j in range(len(i)):
        t = get_word(i[j], eng_tokenizer)
        if j > 0 and (t == get_word(i[j-1], eng_tokenizer) or t is None):
            temp.append('')
        else:
            temp.append(t if t is not None else '')
    preds_text.append(' '.join(temp))

preds_df = pd.DataFrame({'actuals': test[:, 0], 'predicted': preds_text})
print(preds_df.sample(15))

NameError: name 'preds' is not defined

In [ ]:
def define_model(in_vocab, out_vocab, in_timesteps, out_timesteps, units):
    model = Sequential([
        Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True),
        LSTM(units, recurrent_activation='sigmoid'),  # Force standard LSTM
        RepeatVector(out_timesteps),
        LSTM(units, return_sequences=True, recurrent_activation='sigmoid'),
        Dense(out_vocab, activation='softmax')
    ])
    return model